我们使用最普通的完整retrieval方法，因为CSN的数据量尚允许；注意code search和IR（其子集是semantic search）数据类型有所不同

数据集使用CodeSearchNet；测试在数据类型不同且模型不同时，fusion layer的重要性

由于训练数据必须是text并且必须使用bi-encoder，因此sentence-transformer只作为encoder部分（暂定text encoder不训练否则可能学习率不兼容）

最主要比较的是GNN模型的选择，以及和baseline的比较

注：如果数据集大于RAM大小，可以使用PyTorch的IterableDataset进行online learning，缺点是没有办法shuffle

In [1]:
# Install torch geometric
# !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
# !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
!pip install transformers
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=7c457c16a186fdbd52c1e06fe626862d8fefd9afac25d61310adeb490c1a48ea
  Stored in directory: /root/.cache/pip/wheels/cd/7d/6b/17150450b80b4a3656a84330e22709ccd8dc0f8f4773ba4133
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org

In [2]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!unzip python.zip
!gunzip ./python/final/jsonl/train/python_train_*.jsonl.gz
!gunzip ./python/final/jsonl/valid/python_valid_*.jsonl.gz
!gunzip ./python/final/jsonl/test/python_test_*.jsonl.gz

--2023-04-14 16:04:12--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.11.46, 52.216.54.232, 52.217.133.152, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.11.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  13.2MB/s    in 71s     

2023-04-14 16:05:24 (12.7 MB/s) - ‘python.zip’ saved [940909997/940909997]

Archive:  python.zip
   creating: python/
   creating: python/final/
   creating: python/final/jsonl/
   creating: python/final/jsonl/train/
  inflating: python/final/jsonl/train/python_train_9.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_12.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_10.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_0.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_6.js

## Cleaning CodeSearNet dataset

In [3]:
import os
import json
import ast

for root, dirs, files in os.walk("./python/final/jsonl"):
  for name in files:
    data = []
    file_name = os.path.join(root, name)
    with open(file_name, "r") as f:
      cleaned_name = os.path.splitext(file_name)[0] + "_cleaned" + os.path.splitext(file_name)[1]
      for line in f:
        line = line.strip()
        js = json.loads(line)
        try:
          if 'code' in js:
            code = js['code']
          else:
            code = js['function']
          parsed = ast.parse(code)
        except SyntaxError:
          pass
        else:
          data.append(line + '\n')     
    os.remove(file_name)
    last_line = data.pop(-1)
    data.append(last_line.strip('\n'))
    with open(cleaned_name, "w") as g:
      g.writelines(data)

In [4]:
total_data = ''
for file_name in os.listdir("./python/final/jsonl/train"):
  file_name = os.path.join("./python/final/jsonl/train", file_name)
  with open(file_name, 'r') as f:
    data = f.read()
    total_data += data
    total_data += '\n'
  os.remove(file_name)

with open("./python/final/jsonl/train/python_train_cleaned.jsonl", 'w') as g:
  g.write(total_data.strip('\n'))

## Code AST construction and preprocessing

In [5]:
# https://docs.python.org/3/library/ast.html#ast-helpers
# some of the classes may vary according to the version of Python, e.g. "Match" is not present before 3.10

class_list = [
  "AST",
  "Constant", "FormattedValue", "JoinedStr",
  "List", "Tuple", "Set", "Dict", "Name",
  "Load", "Store", "Del", "Starred", "Expr",
  "UnaryOp", "UAdd", "USub", "Not", "Invert",
  "BinOp", "Add", "Sub", "Mult", "Div", "FloorDiv",
  "Mod", "Pow", "LShift", "RShift", "BitOr",
  "BitXor", "BitAnd", "MatMult", "BoolOp",
  "And", "Or", "Compare", "Eq", "NotEq",
  "Lt", "LtE", "Gt", "GtE", "Is", "IsNot",
  "In", "NotIn", "Call", "keyword", "IfExp",
  "Attribute", "NamedExpr", "Subscript", "Slice",
  "ListComp", "SetComp", "GeneratorExp", "DictComp",
  "comprehension", "Assign", "AnnAssign", "AugAssign",
  "Raise", "Assert", "Delete", "Pass", "Import",
  "ImportFrom", "alias", "If", "For", "While",
  "Break", "Continue", "Try", "TryStar", "ExceptHandler",
  "With", "withitem", "Match", "match_case",
  "MatchValue", "MatchSingleton", "MatchSequence",
  "MatchStar", "MatchMapping", "MatchClass",
  "MatchAs", "MatchOr", "FunctionDef", "Lambda",
  "arguments", "arg", "Return", "Yield", "YieldFrom",
  "Global", "Nonlocal", "ClassDef", "AsyncFunctionDef",
  "Await", "AsyncFor", "AsyncWith"
]

class_dict = {class_list[i]: i for i in range(len(class_list))}

In [6]:
# based on https://gist.github.com/joshmarlow/4001898
# AST到NetworkX的转化，NetworkX到PyG图的转化，以及节点特征的构建
import ast
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer

class viz_walker(ast.NodeVisitor):
  def __init__(self):
    self.stack = []
    self.graph = nx.Graph()
    self.vectorizer = CountVectorizer(lowercase=False, vocabulary=class_dict, binary=True)

  def generic_visit(self, stmt):
    node_name = str(stmt)

    parent_name = None

    if self.stack:
      parent_name = self.stack[-1]

    self.stack.append(node_name)

    self.graph.add_node(node_name, x=self.vectorizer.transform([stmt.__class__.__name__]).toarray().astype(np.float32).squeeze())

    if parent_name:
      self.graph.add_edge(node_name, parent_name)

    super(self.__class__, self).generic_visit(stmt)

    self.stack.pop()

## GNN model as code encoder

In [7]:
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)

import os

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import from_networkx

2.0.0+cu118


In [8]:
# based on codebase of CS224W
class GCN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, return_embeds=False):
    super(GCN, self).__init__()

    # A list of GCNConv layers
    self.convs = nn.ModuleList()

    # A list of 1D batch normalization layers
    self.bns = nn.ModuleList()

    # The log softmax layer
    self.softmax = nn.LogSoftmax(dim=-1)

    self.convs.append(GCNConv(input_dim, hidden_dim))
    for _ in range(num_layers - 2):
      self.convs.append(GCNConv(hidden_dim, hidden_dim))
    self.convs.append(GCNConv(hidden_dim, output_dim))

    for _ in range(num_layers - 1):
      self.bns.append(nn.BatchNorm1d(hidden_dim))

    self.relu = nn.ReLU()

    # Probability of an element getting zeroed
    self.dropout = nn.Dropout(p=dropout)

    # Skip classification layer and return node embeddings
    self.return_embeds = return_embeds

  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()
    for bn in self.bns:
      bn.reset_parameters()

  def forward(self, x, adj_t):
    for i in range(len(self.convs) - 1):
      x = self.convs[i](x, adj_t) # 可以是edge list也可以是adjacency matrix
      x = self.bns[i](x)
      x = self.relu(x)
      x = self.dropout(x)

    out = self.convs[-1](x, adj_t) # 如果输出embedding，那么维度是output_dim
    if self.return_embeds == False:
      out = self.softmax(out)

    return out

In [9]:
### GCN to predict graph property
# based on codebase of CS224W
class GCN_Graph(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, dropout):
    super(GCN_Graph, self).__init__()

    # Node embedding model
    self.gnn_node = GCN(input_dim, hidden_dim,
      hidden_dim, num_layers, dropout, return_embeds=True)

    self.pool = global_mean_pool # 从节点embedding出发，用mean pooling得到graph embedding

  def reset_parameters(self):
    self.gnn_node.reset_parameters()
    self.linear.reset_parameters()

  def forward(self, batched_data):
    x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch

    out = self.gnn_node(x, edge_index)
    out = self.pool(out, batch)

    return out

## The whole retrieval model

In [10]:
class Model(nn.Module):  
  def __init__(self, text_encoder, code_encoder):
    super(Model, self).__init__()
    self.text_encoder = text_encoder
    self.code_encoder = code_encoder
    self.loss_fn = nn.CrossEntropyLoss()

    
  def forward(self, code_inputs, nl_inputs, return_vec=False): 
    nl_vec = self.text_encoder.encode(nl_inputs, convert_to_tensor=True)
    code_vec = self.code_encoder(code_inputs)
    if return_vec:
      return code_vec, nl_vec

    scores = (nl_vec[:, None, :] * code_vec[None, :, :]).sum(-1) # dot product of nl and code vectors as the score
    loss = self.loss_fn(scores, torch.arange(64, device=scores.device))
    return loss, code_vec, nl_vec

## Data and model preparation

In [11]:
import random

import numpy as np
from torch.utils.data import Dataset

from torch_geometric.loader import DataLoader

import json

from tqdm import tqdm

from transformers import AdamW, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer

In [12]:
class TextDataset(Dataset):
  def __init__(self, data_path=None):
    self.code = []
    self.query = []
    with open(data_path, 'r') as f:
      for line in f:
        line = line.strip()
        js = json.loads(line)
        if 'code' in js:
          self.code.append(js['code'])
        else:
          self.code.append(js['function'])
        self.query.append(js['docstring'])

  def __len__(self):
    return len(self.code)

  def __getitem__(self, i):
    parsed = ast.parse(self.code[i])
    nw = viz_walker()
    nw.visit(parsed)
    code_data = from_networkx(nw.graph)
    return code_data, self.query[i]


def set_seed(seed=45):
  random.seed(seed)
  os.environ['PYHTONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

In [13]:
def evaluate(model):
    print("Running evaluation...")
    model.eval()
    code_vecs = [] 
    nl_vecs = []
    for batch in eval_dataloader:
        code_inputs = batch[0].to(device)    
        nl_inputs = batch[1]
        with torch.no_grad():
            loss, code_vec, nl_vec = model(code_inputs, nl_inputs)
            code_vecs.append(code_vec.cpu().numpy())
            nl_vecs.append(nl_vec.cpu().numpy())
    code_vecs = np.concatenate(code_vecs, 0)
    nl_vecs = np.concatenate(nl_vecs, 0)

    scores = np.matmul(nl_vecs, code_vecs.T)

    ranks = []
    for i in range(len(scores)):
        score = scores[i,i]
        rank = 1
        for j in range(len(scores)):
            if i != j and scores[i, j] >= score:
                rank += 1
        ranks.append(1 / rank)    
    
    mrr = float(np.mean(ranks))

    return mrr

## Training and testing

In [ ]:
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
print(f"using {device}.")

train_data_path = "./python/final/jsonl/train/python_train_cleaned.jsonl"
eval_data_path = "./python/final/jsonl/valid/python_valid_0_cleaned.jsonl"
output_dir = "./outputs"

# set_seed(45)

text_encoder = SentenceTransformer("all-mpnet-base-v2", device=device)
code_encoder = GCN_Graph(input_dim=len(class_dict), hidden_dim=768, num_layers=3, dropout=0.5)
model = Model(text_encoder, code_encoder)
model.to(device)

for param in text_encoder.parameters(): # we first test with fixed text encoder
  param.requires_grad = False

train_dataset = TextDataset(train_data_path)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64)
eval_dataset = TextDataset(eval_data_path)
eval_dataloader = DataLoader(eval_dataset, shuffle=False, batch_size=64)

num_train_epochs = 5
save_steps = 2000

optimizer = AdamW(model.parameters(), lr=0.01)
scheduler = get_linear_schedule_with_warmup(optimizer, len(train_dataloader), len(train_dataloader) * num_train_epochs)

step = 0
best_mrr = 0.0

for idx in range(num_train_epochs):
  for batch in tqdm(train_dataloader):
    code_inputs = batch[0].to(device)
    nl_inputs = batch[1]

    model.train()
    loss, code_vec, nl_vec = model(code_inputs, nl_inputs)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()
    step += 1

    if step % save_steps == 0:
      mrr = evaluate(model)
      print(f"Mrr = {mrr}")
      if mrr >= best_mrr:
        best_mrr = mrr
        print(f"Save checkpoint!")

        checkpoint_prefix = 'checkpoint-best-mrr'
        output_dir = os.path.join(output_dir, checkpoint_prefix)
        if not os.path.exists(output_dir):
          os.makedirs(output_dir)
        model_to_save = model.module if hasattr(model, 'module') else model

        torch.save(model_to_save.state_dict(), os.path.join(output_dir, 'pytorch_model.bin'))

using 0.


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/6378 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch_geometric/utils/convert.py:249: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  data[key] = torch.tensor(value)
  6%|▌         | 367/6378 [11:28<2:46:32,  1.66s/it]

In [ ]:
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
print(f"using {device}.")
 
test_data_file = "./python/final/jsonl/test/python_test_0_cleaned.jsonl"
output_dir = "./outputs"

# set_seed(45)

text_encoder = SentenceTransformer("all-mpnet-base-v2", device=device)
code_encoder = GCN_Graph(input_dim=len(class_dict), hidden_dim=768, num_layers=3, dropout=0.5)
model = Model(text_encoder, code_encoder)
model.load_state_dict(torch.load(os.path.join(output_dir, 'checkpoint-best-mrr/pytorch_model.bin')))
model.to(device)

test_dataset = TextDataset(test_data_file)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=64)

print("Running Test...")
model.eval()
code_vecs = [] 
nl_vecs = []
for batch in eval_dataloader:
    code_inputs = batch[0].to(device)    
    nl_inputs = batch[1]
    with torch.no_grad():
        loss, code_vec, nl_vec = model(code_inputs, nl_inputs)
        code_vecs.append(code_vec.cpu().numpy())
        nl_vecs.append(nl_vec.cpu().numpy())
code_vecs = np.concatenate(code_vecs, 0)
nl_vecs = np.concatenate(nl_vecs, 0)

scores = np.matmul(nl_vecs, code_vecs.T)

sort_ids=np.argsort(scores, axis=-1, kind='quicksort', order=None)[:, ::-1]
indexs = []
urls = []
for example in eval_dataset.examples:
    indexs.append(example.idx)
    urls.append(example.url)
with open(os.path.join(output_dir, "predictions.jsonl"), 'w') as f:
    for index, url, sort_id in zip(indexs, urls, sort_ids):
        js = {}
        js['url'] = url
        js['answers'] = []
        for idx in sort_id[:100]:
            js['answers'].append(indexs[int(idx)])
        f.write(json.dumps(js)+'\n') 